In [23]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,roc_auc_score,classification_report,recall_score,precision_score
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import KFold
import json
from joblib import dump, load
import pymrmr

In [11]:
features_2d = pd.read_csv('./Data/3D_t2/all.csv')
data = features_2d.drop(columns=['Center'])

In [12]:

correlation = data.iloc[:,2:].corr(method='spearman')

In [13]:
selected_features = correlation[(correlation.abs() <= 0.6) | (correlation.abs() == 1)]

for i in range(200):
    for index in range(selected_features.shape[0]):
        row = selected_features.iloc[index]
        to_drop = row[row.isna()].index
        selected_features_c_droped = selected_features.drop(labels=to_drop,axis=0)
        selected_features_r_droped = selected_features_c_droped.drop(labels=to_drop,axis=1)
        if selected_features_r_droped.shape[0]<selected_features.shape[0]:
            break
    selected_features = selected_features_r_droped

In [19]:
selected_data = data[selected_features.index.to_list()+['Label','Name']]

In [ ]:
selected_data["Label"] = selected_data['Label'].apply(lambda x: x-1)



C:\Users\sugar\AppData\Local\Temp\ipykernel_9140\4066838598.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data["Label"] = selected_data['Label'].apply(lambda x: x-1)


In [21]:
def get_results(acc_list,auc_list,recall_list,precision_list):    
    acc_list = np.array(acc_list)
    auc_list = np.array(auc_list)
    recall_list = np.array(recall_list)
    precision_list = np.array(precision_list)
    print(f'Recall, Average:{recall_list.mean():.4f}, Std:{recall_list.std():.4f}')
    print(f'precision, Average:{precision_list.mean():.4f}, Std:{precision_list.std():.4f}')
    print(f'Accuracy, Average:{acc_list.mean():.4f}, Std:{acc_list.std():.4f}')
    print(f'AUC, Average:{auc_list.mean():.4f}, Std:{auc_list.std():.4f}')

In [ ]:
val_acc_list = []
val_auc_list = []
val_recall_list = []
val_precision_list = []

test_acc_list = []
test_auc_list = []
test_recall_list = []
test_precision_list = []



for fold in range(1,6):
    with open("Fold_data.json", "r") as file:
        fold_data = json.load(file)
    # Radiomics features for training and testing
    train_list = fold_data[f'Fold{fold}']['train']
    test_list = fold_data[f'Fold{fold}']['test']
    train_data = selected_data[selected_data["Name"].isin(train_list)]
    test_data = selected_data[selected_data["Name"].isin(test_list)]
    
    # print(train_data.shape)
    train_y = train_data[['Label']]
    train_x = train_data.iloc[:,:-2]

    test_y = test_data[['Label']]
    test_x = test_data.iloc[:,:-2]

    standard_scaler = StandardScaler()
    train_x = standard_scaler.fit_transform(train_x)
    test_x = standard_scaler.transform(test_x)

    model = RandomForestClassifier(n_estimators=200, random_state=42,max_depth=20)
    model.fit(train_x, train_y)

    y_pred_proba = model.predict(test_x)  # Get probabilities for the positive class
    y_pred = model.predict(test_x)

    accuracy = accuracy_score(test_y, y_pred)
    # print(f'Accuracy: {accuracy:.4f}')
    auc_score = roc_auc_score(test_y, y_pred_proba)
    # print(f'AUC Score: {auc_score:.4f}')
    # Print the classification report
    recall = recall_score(test_y, y_pred)
    # print(f'Recall Score: {recall:.4f}')
    precision = precision_score(test_y, y_pred)
    # print(f'AUC Score: {precision:.4f}')

    val_acc_list.append(accuracy)
    val_auc_list.append(auc_score)
    val_recall_list.append(recall)
    val_precision_list.append(precision)
    # Save the model
    dump(model, f'./RFmodels_3d/RF_{fold}.joblib')
    
print('Validation set')
get_results(val_acc_list,val_auc_list,val_recall_list,val_precision_list)
# print('--------------------')
# print("Test set")
# get_results(test_acc_list,test_auc_list,test_recall_list,test_precision_list)
print('================================================')

f:\Anaconda3\envs\monai\Lib\site-packages\sklearn\base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
f:\Anaconda3\envs\monai\Lib\site-packages\sklearn\base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
f:\Anaconda3\envs\monai\Lib\site-packages\sklearn\base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
f:\Anaconda3\envs\monai\Lib\site-packages\sklearn\base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for exa

Validation set
Recall, Average:0.4742, Std:0.1222
precision, Average:0.5974, Std:0.0769
Accuracy, Average:0.6654, Std:0.0546
AUC, Average:0.6342, Std:0.0610
